# Machine Learning

Aqui aplicaremos técnicas de machine learning ao dataset de uso da força do departamento de polícia de Seattle.

In [53]:
#encoding:utf-8
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

In [54]:
data = pd.read_csv('./data/use-of-force.csv', sep=",").set_index('ID');

# Atributos extraídos da data

O campo **Occured_date_time** possui informações sobre a data e a hora da ocorrência. Dele podemos extrair uma série de atributos relevantes:

1. **dia:** o período do mês em que o incidente ocorreu. Não sabemos como isso afeta a possibilidade de um evento level 2, então vamos deixar o classificador decidir

2. **mês:** o período do ano em que o incidente ocorreu. Será que, por exemplo, nos meses de inverno as ocorrências level 2 são proporcionalmente mais frequentes?

3. **hora:** o período do dia em que o incidente ocorreu. Já vimos que o número de ocorrências varia ao longo do dia. Será que isso ajuda a definir o tipo de ocorrência?

Atributos que decidimos não usar:

1. **ano:** o classificador poderia aprender que, por exemplo, o ano de 2016 tem proporcionalmente mais ocorrências level 2 que o ano de 2017. Isso, entretanto, não é relevante pra gente, pois nunca mais teremos uma nova ocorrência em 2016. Ou seja, saber que 2016 tem proporcionalmente mais chances de ocorrências level 2 não vai ajudar o classificador a rotular as amostras de 2019. Além disso, o classificador vai ter sido treinado sem nenhuma amostra de 2019, por tanto não dá pra prever como ele se comportaria em relação a isso. Poderiamos ter um caso de overfiting nas ocorrências dos anos que apresentamos.

2. **dia da semana:** poderiamos extrair o dia da semana da data e usa-lo como um novo atributo. Porém, como vimos na nossa análise dos dados que as ocorrências se distribuem mais ou menos da mesma forma em relação aos dias da semana, decidimos ignorar esse dado.

3. **minutos, segundos:** por acreditar que são características quase aleatórias, iremos ignora-las.

In [55]:
data['day'] = [pd.to_datetime(D, yearfirst=True).day for D in data['Occured_date_time']];
data['month'] = [pd.to_datetime(D, yearfirst=True).month for D in data['Occured_date_time']];
data['hour'] = [pd.to_datetime(D, yearfirst=True).hour for D in data['Occured_date_time']];

# Tratamento dos dados faltantes

Algumas colunas do dataset possuem dados faltantes (missing data). Elas aparecem na coluna das delegacias, dos setores, dos beats, da raça e do gênero. Todas as colunas com missing data são categóricas.

## De onde vem a ausencia dos dados?

Dados faltantes podem ser aleatórios ou não aleatórios. Dizemos que o dado é faltante não aleatório quando o fato da informação não estar preenchida é, em si, relevante. Por exemplo, Imagine que os policiais de uma determinada delegacia estivessem evitando de preencher o campo "Precinct" para ficar com uma estatística mais favorável. Nesse caso, o fato do campo "Precinct" não estar preenchido seria um indicio de que a ocorrência foi naquela delegacia. Apenas analisando os dados, não é possível afirmar se os dados faltantes são aleatórios ou não. Assim, seria interessante buscar uma técnica que funcione relativamente bem nos dois casos.

## Por que não adotar a abordagem mais simples?

A forma mais simples de tratar missing data é remover todas as amostras com dados faltantes. Normalmente buscamos essa estratégia quando o número de amostras incompletas não é muito grande, pois isso tornaria o dataset resultante pequeno, o que poderia inviabilizar a construção de um bom classificador. No nosso caso o dataset formado apenas por amostras sem missing data ainda teria um tamanho administrável, porém eles estão presentes em uma quantidade significativa dos registros. Considerando que a amostra que possuimos é representativa, e em machine learning nós sempre fazemos essa suposição, então o nosso classificador será submetido ao trabalho de classificar dados faltantes com frequencia não despresível. A forma como vamos tratar missing data deve levar isso em consideração, por isso, a estratégia mais simples de remover as linhas com missing data do treinamento nem será considerada.

## Estratégia adotada

Neste trabalho, consideraremos que a ausência dos dados é uma nova classe de valores possíveis para aquele atribulto. Como em uma das classes, alguns registros possuem o valor **'-'** informado, adotaremos esse padrão para todos. Assim, o campo gênero, por exemplo, poderá ter os valores: **Masculino**, **Feminino** e **-**. 

In [56]:
imp = SimpleImputer(strategy="constant", fill_value='-');
data = imp.fit_transform(data);
[N, D] = np.shape(data);

# Codificação dos padrões

Atributos categóricos devem ser tratados de forma diferente dos ordinais. Se pensarmos no atribulto cor, por exemplo, se uma amostra possui a cor amarela e eu digo que é vermelha, isso não é um erro maior nem menor que dizer que é azul. Ambos os erros são equivalentes. Assim, os atributos categóricos devem ser modelados de forma que entre dois valores diferentes quaisquer, a distância entre eles seja sempre a mesma. A forma mais comum de se atingir esse objetivo é usando **One hot encoding**.

Nessa codificação, para cada possível valor de um atributo é criada uma coluna no dataset resultante. A coluna correspondente ao valor da amostra é preenchido com **1** e as demais são preenchidas com **0** (daí o nome **one hot**). Por exemplo, se uma coluna pode ter os valores **A**, **B** e **C**, serão criadas 3 colunas. Uma amostra cujo atributo seja **A** será mapeada no vetor **[1,0,0]**, uma amostra com valor **B** será mapeada em **[0,1,0]** e uma amostra do valor **C** terá o vetor **[0,0,1]**. Note que os vetores onde o atributo é mapeado são equidistantes.

## Codificação do nosso banco

A maioria dos nossos atributos é categórica. A data e os campos que derivamos dela poderiam ser vistos como ordinais. A justificativa para isso seria que o mês 5 é mais próximo do mês 6 do que do mês 11, por exemplo. Não usaremos essa representação por dois motivos:
1. as datas são cíclicas, então o mês 11 está mais próximo do mês 1 do que do mês 4
2. estamos ignorando o ano, então o mês 5 de 2016 está mais próximo do mês 9 de 2016 do que do mês 5 de 2019

Para evitar que o classificador leve em conta uma ordenação inválida, trataremos os campos relacionados a data também como categóricos. O nosso dataset é numeroso o bastante para que cada categoria tenha uma quantidade significativa de registros. 

In [57]:
X = data[:,[1,3,4,5,6,7,8,9,10,11,12]];
enc = OneHotEncoder();
X = enc.fit_transform(X);
X = X[:,2:];
T = [int(w[6]) for w in data[:,1]];

# Classificação

Faremos a classificação usando um comitê de SVMs. Usaremos kernel RBF e a variabilidade dos modelos individuais será obtida passando para cada classificador um subconjunto distinto do dataset.  

In [58]:

clf = BaggingClassifier(svm.SVC(kernel='rbf', C=0.1), max_samples=0.5, max_features=0.8, n_estimators=10,bootstrap_features=True);

scores = cross_val_score(clf, X, T, cv=5);
print(scores.mean());

0.7640491976822412
